In [3]:
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [27]:
# Generate small Kaggle-Dataset
df_kaggle = pd.read_csv("..\\data\\mitbih_test.csv", header=None)
X_kaggle = df_kaggle.iloc[:, :-1]
y_kaggle = df_kaggle.iloc[:, -1]

rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_kaggle, y_kaggle)
df_kaggle_resampled = pd.concat([X_resampled, y_resampled], axis=1).reset_index()
if False:
    df_kaggle_resampled.to_pickle("..\\reports\\Streamlit\\data\\Resampled_kaggle_data")

66

In [41]:
# Generate small original-Dataset
class_labels = {"N": 0,
                "S": 1,
                "V": 2,
                "F": 3,
                "Q": 4}

df_mit = pd.read_pickle("..\\data\\.MIT_Heartbeat_original_fs180hz.pkl")
X_mit = df_mit.iloc[:, 6:]
y_mit = df_mit["class"].replace(class_labels)
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_mit, y_mit)
# Limit to 162 values per class (same as the kaggle-downsampled dataset)
y_resampled = y_resampled.iloc[710-162:]
X_resampled = X_resampled.iloc[710-162:, :]
X_resampled, y_resampled = rus.fit_resample(X_resampled, y_resampled)
df_mit_resampled = pd.concat([X_resampled, y_resampled], axis=1).reset_index()
df_mit_resampled.drop("index", axis=1, inplace=True)
if False:
    df_mit_resampled.to_pickle("..\\reports\\Streamlit\\data\\Resampled_original_data")

C:\Users\felix\AppData\Local\Temp\ipykernel_37308\2856043661.py:10: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [50]:
nullvalues = (df_mit.iloc[:, 6:] == 0).sum().sum()

notnullvalues = (df_mit.iloc[:, 6:] != 1000).sum().sum()

nullvalues/notnullvalues

0.5519132362666551

In [38]:
df = pd.read_pickle("..\\reports\\Streamlit\\data\\Resampled_original_data")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

time = np.arange(0, 310/180, 1/180)
i = np.random.randint(0, 161)
class_index = [0, 162, 324, 486, 648]
colors = ['red', 'blue', 'green', 'orange', 'purple']

fig = make_subplots(rows=2, cols=1, subplot_titles=("Channel L2", "Channel V2"),
                    vertical_spacing = 0.1)
for cat in [0, 1, 2, 3, 4]:
    fig.add_trace(go.Scatter(x=time, y=df.iloc[class_index[cat] + i, 0:310], name=f"Class {cat}",
                             line=dict(color=colors[cat]), legendgroup = str(cat)),
                    row=1, col=1)
    fig.add_trace(go.Scatter(x=time, y=df.iloc[class_index[cat] + i, 310:-1], name=f"Class {cat}",
                             line=dict(color=colors[cat]), legendgroup = str(cat)),
                    row=2, col=1)
fig.update_layout(title='Random examples from Kaggle-Dataset',
                  xaxis2_title='Time / s',
                  yaxis1_title='ECG-Channel L2 / a.u.',
                  yaxis2_title='ECG-Channel V5 / a.u.',
                  legend_title="Arrythmia",
                  template = "ggplot2",
                  width=750,
                  height=600,
                  margin=dict(l=80, r=50, t=75, b=50))
for i, yaxis in enumerate(fig.select_yaxes(), 1):
    legend_name = f"legend{i}"
    fig.update_layout({legend_name: dict(y=yaxis.domain[1], yanchor="top")}, showlegend=True)
    fig.update_traces(row=i, legend=legend_name)
fig.show()

In [40]:
df.iloc[1, 315:316]

v5_0    0.0
Name: 1, dtype: float64

In [35]:
# Generate Class-Diagram:
import pandas as pd
import plotly.graph_objects as go
class_labels = {"N": 0,
                "S": 1,
                "V": 2,
                "F": 3,
                "Q": 4}
labels_long = {0: "Normal beat",
               1: "Supraventricular premature beat",
               2: "Premature ventricular contraction",
               3: "Fusion of ventricular and normal beat",
               4: "Unclassifiable beat"}

df = pd.read_pickle("..\\data\\.MIT_Heartbeat_original_fs180hz.pkl")
y= df["class"].replace(class_labels)
classes = y.value_counts().sort_index()

if False:
    classes.to_pickle("..\\reports\\Streamlit\\data\\MIT_classdistribution")

class_distribution = pd.read_pickle("data\\MIT_classdistribution")
custom_labels = [f'Class {i}: {labels_long[i]}' for i in classes.index]
colors = ['red', 'blue', 'green', 'orange', 'purple']

fig = go.Figure(go.Pie(labels=custom_labels, values=class_distribution.values, marker=dict(colors=colors)
                       ))
fig.update_layout(title='Class Distribution',
                  legend_title="Classes",
                  template = "ggplot2",
                  width=600,
                  height=300,)
fig.show()



C:\Users\felix\AppData\Local\Temp\ipykernel_37308\3363734199.py:16: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [1]:
df = pd.read_pickle("..\\reports\\Streamlit\\data\\Resampled_kaggle_data")

import plotly.express as px
import plotly.graph_objects as go

time = np.arange(0, 187/125, 1/125)
i = np.random.randint(0, 161)
class_index = [0, 162, 324, 486, 648]
fig = go.Figure()
for cat in [0, 1, 2, 3, 4]:
    fig.add_trace(go.Scatter(x=time, y=df.iloc[class_index[cat] + i, 1:-1], name=f"Class {cat}"))
fig.update_layout(title='Random examples from Kaggle-Dataset',
                  xaxis_title='Time / s',
                  yaxis_title='ECG-Values /  arb.unit',
                  legend_title="Arrythmia",
                  template = "ggplot2",
                  width=750,
                  height=400,)
fig.show()
i

NameError: name 'pd' is not defined

In [83]:
import tensorflow as tf

model = tf.keras.models.load_model('..\\models\\combined_cnn1_bs256_k7_noweights.keras', compile=False)
# model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.predict([np.atleast_2d(np.random.rand(315)), np.atleast_2d(np.random.rand(315))])
# model.summary()

1/1 [==============================] - 0s 102ms/step


array([[9.9895942e-01, 1.0054500e-03, 3.1215601e-05, 3.9438228e-06,
        3.8404323e-08]], dtype=float32)

In [89]:
import time
a = time.time()
model.predict([np.atleast_2d(np.random.rand(315)), np.atleast_2d(np.random.rand(315))])
b = time.time()
print(b-a)

1/1 [==============================] - 0s 15ms/step
0.040212392807006836
